# Resnet18

In [ ]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):

    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        #residual function
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class BottleNeck(nn.Module):
    """Residual block for resnet over 50 layers

    """
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class ResNet(nn.Module):

    def __init__(self, block, num_block, num_classes=100):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        #we use a different inputsize than the original paper
        #so conv2_x's stride is 1
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """make resnet layers(by layer i didnt mean this 'layer' was the
        same as a neuron netowork layer, ex. conv layer), one layer may
        contain more than one residual block

        Args:
            block: block type, basic block or bottle neck block
            out_channels: output depth channel number of this layer
            num_blocks: how many blocks per layer
            stride: the stride of the first block of this layer

        Return:
            return a resnet layer
        """

        # we have num_block blocks per layer, the first block
        # could be 1 or 2, other blocks would always be 1
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output

def resnet18():
    """ return a ResNet 18 object
    """
    return ResNet(BasicBlock, [2, 2, 2, 2])


# Training

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset

# Define the data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download the CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training, validation, and test sets
# Adjust the proportions based on your preferences
train_size = int(0.8 * len(train_dataset))
val_size = int(0.1 * len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

batch_size = 64  # Adjust as needed

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
! git clone https://github.com/K-Hooshanfar/pytorch-cifar100

Cloning into 'pytorch-cifar100'...
remote: Enumerating objects: 1043, done.
remote: Counting objects: 100% (1043/1043), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 1043 (delta 658), reused 1008 (delta 645), pack-reused 0
Receiving objects: 100% (1043/1043), 492.58 KiB | 4.44 MiB/s, done.
Resolving deltas: 100% (658/658), done.


In [ ]:
%cd pytorch-cifar100


/content/pytorch-cifar100


In [ ]:
!python train.py -net resnet18 -gpu

2023-12-01 12:43:14.467192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 12:43:14.467261: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 12:43:14.467309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 12:43:16.080052: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100% 169001437/169001437 [00:01<00:00, 98021343.40it/s] 
Extracting ./data/cifar-100-python.tar.gz to ./data
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker pro

# Save weights

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil

source_path = '/content/pytorch-cifar100/checkpoint/resnet18/Friday_01_December_2023_12h_43m_17s/resnet18-200-regular.pth'
destination_path = '/content/gdrive/My Drive/resnet18-200-regular.pth'

shutil.copyfile(source_path, destination_path)


'/content/gdrive/My Drive/resnet18-200-regular.pth'

In [ ]:
model = resnet18()

# Load pre-trained weights
pretrained_weights_path = '/content/pytorch-cifar100/checkpoint/resnet18/Friday_01_December_2023_12h_43m_17s/resnet18-200-regular.pth'
model.load_state_dict(torch.load(pretrained_weights_path))

<All keys matched successfully>

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

# Define data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-100 dataset
cifar100_test = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)


# Create PyTorch data loaders
batch_size = 128
val_loader = torch.utils.data.DataLoader(cifar100_test, batch_size=batch_size, shuffle=False)

# Check the number of samples in each set
print(f"Test set size: {len(cifar100_test)}")

Files already downloaded and verified
Test set size: 10000


In [1]:
model.eval()
correct_test = 0
total_test = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Final Test Accuracy: {test_accuracy:.2f}%")

Final Test Accuracy: 76.28%


# Load Model and Get Feature Latent

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch
from torchvision import datasets, transforms
import random
import numpy as np

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

# Load the CIFAR-100 dataset and create a balanced subset
transform = transforms.Compose([transforms.ToTensor()])
# Load CIFAR-100 dataset
cifar100_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(cifar100_dataset))
val_size = len(cifar100_dataset) - train_size
cifar100_traindataset, cifar100_valdataset = torch.utils.data.random_split(cifar100_dataset, [train_size, val_size])

# Define the subset size
subset_fraction = 0.9
subset_size_train = int(subset_fraction * len(cifar100_traindataset))

class_indices = list(range(len(cifar100_traindataset.dataset.classes)))
class_subset_size = int(subset_size_train / len(cifar100_traindataset.dataset.classes))

class_sampler_indices_train = []

for class_index in class_indices:
    class_indices_list_train = [i for i, label in enumerate(cifar100_traindataset.dataset.targets) if label == class_index]
    class_sampler_indices_train.extend(class_indices_list_train[:class_subset_size])

train_sampler = SubsetRandomSampler(class_sampler_indices_train)

batch_size = 256
train_loader = torch.utils.data.DataLoader(cifar100_traindataset, batch_size=batch_size, sampler=train_sampler)

# Check the number of samples in the balanced train set
print(f"Balanced Train set size: {len(train_loader.sampler)}")

Files already downloaded and verified
Balanced Train set size: 36000


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = resnet18()
model.to(device)

# Load pre-trained weights
pretrained_weights_path = '/content/pytorch-cifar100/checkpoint/resnet18/Friday_01_December_2023_12h_43m_17s/resnet18-200-regular.pth'
model.load_state_dict(torch.load(pretrained_weights_path))

# Remove the fully connected layer
model = nn.Sequential(*(list(model.children())[:-1]))

features = []
labels = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for inputs, targets in tqdm(train_loader):
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

        # Forward pass through the model
        features_batch = model(inputs)

        # Append the extracted features and labels
        features.append(features_batch)  # Assuming you want to save features as numpy arrays
        labels.append(targets)

        # Release GPU memory
        del inputs
        torch.cuda.empty_cache()


100%|██████████| 79/79 [00:13<00:00,  5.88it/s]


In [ ]:
# Stack and reshape the extracted features
features = torch.cat(features)
features = features.view(features.size(0), -1)
labels = torch.cat(labels)
labels = labels.unsqueeze(1)

# Metrics

In [4]:
!git clone https://github.com/Arhosseini77/data_complexity_measures

fatal: destination path 'data_complexity_measures' already exists and is not an empty directory.


In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.utils.data import Subset, DataLoader
import numpy as np
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data.sampler import SubsetRandomSampler

from data_complexity_measures.models.ARH_SeparationIndex import ARH_SeparationIndex

In [ ]:
# Create Instance of class
si_calculator = ARH_SeparationIndex(features, labels, normalize=True)

Data has been normalized


# SI

In [ ]:
si_data = si_calculator.si()
print(si_data)

Calculating SI: 100%|██████████| 20000/20000 [00:01<00:00, 13264.24it/s]

0.4858


# Calc High order SI (order = 2)

In [ ]:
si_high_order_2_data = si_calculator.high_order_si(order=2)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 20000/20000 [00:02<00:00, 8999.25it/s] 

0.3387500047683716


# High order soft SI (order=2)

In [ ]:
si_soft_order_2_data = si_calculator.soft_order_si(order=2)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 20000/20000 [00:01<00:00, 11608.36it/s]

Soft Order(2) SI : 0.46242499351501465


# Center Based SI

In [ ]:
center_si_data = si_calculator.center_si()
print("Center SI:", center_si_data)

Calculating Class Centers: 100%|██████████| 100/100 [00:00<00:00, 5365.28it/s]

Center SI: 0.5596500039100647


# Anti SI (order = 2)

In [ ]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 20000/20000 [00:01<00:00, 12144.45it/s]

anti_si: 0.4138999879360199


# Metrics for test set

In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
# Define data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-100 dataset
cifar100_test = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Create PyTorch data loaders
batch_size = 128
test_loader = torch.utils.data.DataLoader(cifar100_test, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = resnet18()
model.to(device)

# Load pre-trained weights
pretrained_weights_path = '/content/pytorch-cifar100/checkpoint/resnet18/Friday_01_December_2023_12h_43m_17s/resnet18-200-regular.pth'
model.load_state_dict(torch.load(pretrained_weights_path))

# Remove the fully connected layer
model = nn.Sequential(*(list(model.children())[:-1]))

features = []
labels = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for inputs, targets in tqdm(test_loader):
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

        # Forward pass through the model
        features_batch = model(inputs)

        # Append the extracted features and labels
        features.append(features_batch)  # Assuming you want to save features as numpy arrays
        labels.append(targets)

        # Release GPU memory
        del inputs
        torch.cuda.empty_cache()


In [ ]:
# Stack and reshape the extracted features
features = torch.cat(features)
features = features.view(features.size(0), -1)
labels = torch.cat(labels)
labels = labels.unsqueeze(1)

In [ ]:
# Create Instance of class
si_calculator = ARH_SeparationIndex(features, labels, normalize=True)

Data has been normalized


# SI

In [ ]:
si_data = si_calculator.si()
print(si_data)

Calculating SI: 100%|██████████| 10000/10000 [00:00<00:00, 13251.85it/s]

0.599


# Calc High order SI (order = 2)

In [ ]:
si_high_order_2_data = si_calculator.high_order_si(order=2)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 10000/10000 [00:00<00:00, 18445.21it/s]

0.4852999746799469


# High order soft SI (order=2)

In [ ]:
si_soft_order_2_data = si_calculator.soft_order_si(order=2)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 10000/10000 [00:01<00:00, 6000.65it/s]

Soft Order(2) SI : 0.5898500084877014


# Center Based SI

In [ ]:
center_si_data = si_calculator.center_si()
print("Center SI:", center_si_data)

Calculating Class Centers: 100%|██████████| 100/100 [00:00<00:00, 2519.49it/s]

Center SI: 0.7026000022888184


# Anti SI (order = 2)

In [ ]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 10000/10000 [00:00<00:00, 11461.91it/s]

anti_si: 0.30559998750686646


# Pretrained Resnet18

In [3]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100

# Load pre-trained models
resnet18_model = models.resnet18(pretrained=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Remove fully connected layers
resnet18_features = torch.nn.Sequential(*(list(resnet18_model.children())[:-1]))


In [14]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_features(model, dataloader):
    model.eval()
    model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            features.append(outputs.squeeze())
            labels.append(targets)

    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Move models to GPU
resnet18_features.to(device)

resnet18_features, resnet18_labels = extract_features(resnet18_features, train_loader)


100%|██████████| 141/141 [00:05<00:00, 23.73it/s]


In [7]:
si_calculator = ARH_SeparationIndex(resnet18_features, resnet18_labels, normalize=True)

Data has been normalized


# SI

In [10]:
si_data = si_calculator.si_batch(batch_size=2000)
print(si_data)

Calculating SI: 100%|██████████| 18/18 [00:00<00:00, 1816.76it/s]

0.22875000536441803


# Calc High order SI (order = 2)

In [11]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2, batch_size = 2000)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 18/18 [00:01<00:00, 17.40it/s]


0.09644444286823273


# High order soft SI (order=2)

In [12]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2,batch_size=2000)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 18/18 [00:00<00:00, 1097.91it/s]


Soft Order(2) SI : 0.2048194408416748


# Center Based SI

In [13]:
center_si_data = si_calculator.center_si_batch(batch_size=2000)
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 18/18 [00:00<00:00, 2524.58it/s]

Center SI: 0.234333336353302


# Anti SI (order = 2)

In [8]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 8000/8000 [00:00<00:00, 14096.75it/s]

anti_si: 0.7402500510215759


Test

In [5]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_features(model, dataloader):
    model.eval()
    model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            features.append(outputs.squeeze())
            labels.append(targets)

    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Move models to GPU
resnet18_features.to(device)

resnet18_features, resnet18_labels = extract_features(resnet18_features, test_loader)


100%|██████████| 79/79 [00:04<00:00, 19.63it/s]


In [6]:
si_calculator = ARH_SeparationIndex(resnet18_features, resnet18_labels, normalize=True)

Data has been normalized


In [7]:
si_data = si_calculator.si_batch(batch_size=2000)
print(":")
print(si_data)

Calculating SI: 100%|██████████| 5/5 [00:00<00:00, 3038.47it/s]

:
0.21709999442100525


In [8]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2, batch_size = 2000)
print(":")
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 5/5 [00:00<00:00, 126.27it/s]

:
0.08919999748468399


In [9]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2,batch_size=2000)
print(":")
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 5/5 [00:00<00:00, 700.80it/s]

:
Soft Order(2) SI : 0.19589999318122864


In [10]:
center_si_data = si_calculator.center_si_batch(batch_size=2000)
print(":")
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 5/5 [00:00<00:00, 1171.33it/s]

:
Center SI: 0.29089999198913574


In [11]:
anti_si = si_calculator.anti_si(order=2)
print(":")
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 10000/10000 [00:01<00:00, 9306.41it/s]

:
anti_si: 0.6976000070571899
